# MSHTA Spawning Windows Shell
Detects a Windows command line executable started from MSHTA.

## Rule Content
```
- title: MSHTA Spawning Windows Shell
  id: 03cc0c25-389f-4bf8-b48d-11878079f1ca
  status: experimental
  description: Detects a Windows command line executable started from MSHTA.
  references:
  - https://www.trustedsec.com/july-2015/malicious-htas/
  author: Michael Haag
  logsource:
    category: process_creation
    product: windows
    service: null
  detection:
    selection:
      ParentImage: '*\mshta.exe'
      Image:
      - '*\cmd.exe'
      - '*\powershell.exe'
      - '*\wscript.exe'
      - '*\cscript.exe'
      - '*\sh.exe'
      - '*\bash.exe'
      - '*\reg.exe'
      - '*\regsvr32.exe'
      - '*\BITSADMIN*'
    condition: selection
  fields:
  - CommandLine
  - ParentCommandLine
  tags:
  - attack.defense_evasion
  - attack.execution
  - attack.t1170
  - car.2013-02-003
  - car.2013-03-001
  - car.2014-04-003
  falsepositives:
  - Printer software / driver installations
  - HP software
  level: high

```

## Querying Elasticsearch

### Import Libraries

In [ ]:
from elasticsearch import Elasticsearch
from elasticsearch_dsl import Search
import pandas as pd

### Initialize Elasticsearch client

In [ ]:
es = Elasticsearch(['http://helk-elasticsearch:9200'])
searchContext = Search(using=es, index='logs-*', doc_type='doc')

### Run Elasticsearch Query

In [ ]:
s = searchContext.query('query_string', query='(process_parent_path.keyword:*\\mshta.exe AND process_path.keyword:(*\\cmd.exe OR *\\powershell.exe OR *\\wscript.exe OR *\\cscript.exe OR *\\sh.exe OR *\\bash.exe OR *\\reg.exe OR *\\regsvr32.exe OR *\\BITSADMIN*))')
response = s.execute()
if response.success():
    df = pd.DataFrame((d.to_dict() for d in s.scan()))

### Show Results

In [ ]:
df.head()